# Task for Today
***  

## Marketing Effectiveness Prediction

Given data about subjects' responses to a bank's marketing campaign, let's try to predict whether a given subject will place a deposit or not.
We will use TensorFlow ANN to make our predictions

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
tf.random.set_seed(100)

In [ ]:
data = pd.read_csv('/kaggle/input/bank-marketing-campaigns-dataset/bank-additional-full.csv', delimiter=';')

In [ ]:
pd.set_option('max_columns', None)

In [ ]:
data

In [ ]:
data.info()

## Encoding Labels

In [ ]:
data.y = data.y.apply(lambda x: 1 if x == 'yes' else 0)

In [ ]:
data.y.sum()

## Encoding Categorical, Sequential, Binary Features

In [ ]:
data.select_dtypes('object')

In [ ]:
{column: len(data[column].unique()) for column in data.select_dtypes('object').columns}

In [ ]:
{column: list(data[column].unique()) for column in data.select_dtypes('object').columns}

In [ ]:
data.replace('unknown', np.nan, inplace=True)

In [ ]:
# Encoding Helper Functions

def encode_onehot(df, columns, prefixes):
    df = df.copy()
    for column, prefix in zip(columns, prefixes):
        dumdums = pd.get_dummies(df[column], prefix=prefix)
        df = pd.concat([df, dumdums], axis=1)
        df = df.drop(column, axis=1)        
    return df

def encode_ordinal(df, columns, orderings):
    df = df.copy()
    for column, ordering in zip(columns, orderings):
        df[column] = df[column].apply(lambda x: ordering.index(x))
    return df

def encode_binary(df, columns, positive_values):
    df = df.copy()
    for column, positive_value in zip(columns, positive_values):
        df[column] = df[column].apply(lambda x: 1 if x == positive_value else x)
        df[column] = df[column].apply(lambda x: 0 if str(x) != 'nan' else x)
    return df

In [ ]:
nominal_features = [
    'job',
    'marital',
    'education',
    'day_of_week',
    'poutcome'
]

ordinal_features = [
    'month'
]

binary_features = [
    'default',
    'housing',
    'loan',
    'contact'
]

In [ ]:
prefixes = ['J', 'M', 'E', 'D', 'P']

orderings = [
    ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
]

positive_values = [
    'yes',    
    'yes',
    'yes',
    'cellular'
]

In [ ]:
data1 = data.copy()
data1 = encode_onehot(data1, nominal_features, prefixes)
data1 = encode_ordinal(data1, ordinal_features, orderings)
data1 = encode_binary(data1, binary_features, positive_values)
data1

## Missing Values Sanity Check

In [ ]:
for column in binary_features:
    data1[column]=data1[column].fillna(data1[column].mean())

In [ ]:
print('Remaining Missing Values: ', data1.isna().sum().sum())

## Split and Scale the Data

In [ ]:
y = data1.y
X = data1.drop('y', axis=1)
X, y

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.7, random_state=123)

## Modeling / Training

In [ ]:
print('Positive Examples: {}'.format(y.sum()))
print('Negative Examples: {}'.format(len(y) - y.sum()))

print("\nClass Distribution: {:.1f} / {:.1f}".format(y.mean() * 100, (1-y.mean())*100) )

In [ ]:
X.shape

In [ ]:
inputs = tf.keras.Input(shape=(X.shape[1]))
#Create 2 hidden layers - 64 activations each - relu activ. fucn.
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.AUC(name='auc')
    ]
)

batch_size=32
epochs = 100

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size = batch_size,
    epochs = epochs,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

## Results

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
y_true = np.array(y_test)
y_pred = np.squeeze(np.array(model.predict(X_test) >= 0.9, dtype=np.int))

In [ ]:
print("Confusion Matrix: \n ", confusion_matrix(y_true, y_pred))

In [ ]:
print("Classification Report: \n", classification_report(y_true, y_pred))